In [ ]:
# Code ported from laptop onto 10.12.68.72 starting on 8/24/2020 (Gregory Rouze)

# To-do: 
# 1) need to separate user functions and main code - I have done this successfully in the offline version, but I'm having a
# little more trouble in the cloud version
# 2) Add comments on putpose of individual user functions
# 3) Run a long term experiment this weekend to see if I run into VPN problems (processing will take a while)


In [ ]:
'''Import relevant packages, functions, and user functions used in this reference ET derivation'''

import boto3
from contextlib import contextmanager 
import earthpy.spatial as es
import fsspec
from math import e
import rasterio as rio
import xarray as xr
from osgeo.gdalnumeric import *
from osgeo.gdalconst import *
import os
from osgeo import gdal, osr, gdal_array, gdalconst
import pandas as pd
import re
import numpy as np
import sys
import ogr
from rasterio import Affine, MemoryFile
from rasterio.enums import Resampling
import rioxarray
from shapely.geometry import Point, Polygon
import geopandas as gpd
from shapely.geometry import box
from fiona.crs import from_epsg
from matplotlib import pyplot as plt
from rasterio.plot import plotting_extent
import earthpy.plot as ep
import math
from itertools import chain
from ipynb.fs.full.ProjectedReferenceET_Classes_Functions import ET0_PM, aggregate_raster_inmem, resample_raster_write, \
reproject_raster, grepfxn, rastermath, lapply_brick, write_geotiff, atmospheric_pressure, relative_fromspecific, unique, s3_push_delete_local
import boto3
# unique # test the import worked correctly

os.getcwd() # home/jupyter-rouze' is root or default path
os.chdir('/home/jupyter-rouze')
# os.chdir('in/temp')
# os.getcwd()

In [ ]:
'''Read configuration file and parse out the inputs line by line'''

configurationfile = 'configurationfile_referenceET_test_future.ini'
# configurationfile = 'configurationfile_referenceET_test_historical.ini'

with open(configurationfile) as f:
    data = {}
    for line in f:
        key, value = line.strip().split(' = ')
        data[key] = value
        
# print(data)

model_files = data['model_files']
data_source = data['data_source']
output_folder = data['output_folder']
elevfile = data['elevfile']
tiffolder = data['tiffolder']
ET0_method = data['ET0_method']
ET0_winddat = data['ET0_winddat']
ET0_crop = data['ET0_crop']
to_clip = data['to_clip']
model = data['model']
northmost = float(data['northmost'])
southmost = float(data['southmost'])
westmost = float(data['westmost'])
eastmost = float(data['eastmost'])
pad_factor = float(data['pad_factor'])
rcp_source = data['rcp_source']
MACA_start_bucket = data['MACA_start_bucket']

'''This is needed to retrieve the netCDF files from the dev-et-data AWS bucket'''
# os.chdir(model_files)
fs = fsspec.filesystem(model_files, anon=False, requester_pays=True)

# list all of the inputs - assumption: this folder only contains input variables.
# model_filesos.listdir()
# print(data_source)
# all_files = fs.find('dev-et-data/in/DelawareRiverBasin/')
all_files = fs.find(MACA_start_bucket)
print(data)
# print(data['elevfile'])

print(all_files)

In [ ]:
# THE CODE BELOW IS PARSED FROM THE CONDIITION WHEN DEALING WITH METDATA

In [ ]:
# Split models apart that are to be used for ensemble averaging
models_parsed = [x.strip() for x in model.split(',')]

# Whittle down the number of files if the folder contains both rcp 4.5 and rcp 8.5 files
# Right now, the code can only handle one model of METDATA output (8/21/2020)
rcp_all_files = [grepfxn(rcp_source,all_files)][0]
# historical_all_files = [grepfxn('historical',all_files)][0]
# print(rcp_all_files)
# print(historical_all_files)

# Iterate the files by each each specified model
models_list=[]
for i in range(len(models_parsed)):
    model_files_loop = [grepfxn(models_parsed[i],rcp_all_files)][0]
    models_list.append(model_files_loop)
        
# Flatten series of lists into one list
rcp_all_files = list(chain(*models_list))

print(rcp_all_files)

# # Iterate the files by each each specified model
# models_list=[]
# for i in range(len(models_parsed)):
#     model_files_loop = [grepfxn(models_parsed[i],historical_all_files)][0]
#     models_list.append(model_files_loop)
        
# # Flatten series of lists into one list
# historical_all_files = list(chain(*models_list))

# print(historical_all_files)


In [ ]:
# Find and compile the year blocks into a list
dfis=[]
# for out in historical_all_files+rcp_all_files:
for out in rcp_all_files:
    a=out.split('_')
    dfi = a[5]+'_'+a[6]
    dfis.append(dfi)

# print(dfis)
    
# Distill the above list into unique year blocks, as there will be duplicates from 
# multiple climate inputs
year_all=unique(dfis);print(year_all)

# For prototyping only
year_block=0
print(year_all[year_block])
# year_all=year_all[1];print(year_all)

In [ ]:
# This code is piecemeal from that within the for loop below
# it was more for me and prototyping within the cloud, so that 
# the variables are the same as those when I created this code offline 
# in other words, I wanted to ensure consistency (Gregory Rouze) - 9/10/2020

# print(rcp_all_files)

# print(year_all[year_block])

# year_block=0
# year_block_files = grepfxn(year_all[year_block],rcp_all_files)
# print(year_block_files)

# year_block=1
# year_block_files = grepfxn(year_all[year_block],rcp_all_files)
# print(year_block_files)

# bounds=[southmost,northmost,westmost,eastmost]
# rcp_pr = lapply_brick(grepfxn("pr",year_block_files), 'precipitation', model_files,tiffolder,data_source,to_clip=to_clip,bounds=bounds,pad_factor=pad_factor)

# print(rcp_pr)

# # Skip 2006-2010 for now because of corrupted files
# # year_all2=year_all[1:] # future
# year_all2=year_all[0:] # historical
# print(year_all2)

# year_block_files = grepfxn(year_all[year_block],rcp_all_files)
# print(year_block_files)

# print(rcp_pr[0][0].count)
# print(doy_loop)
# year_loop

# print(year_all)
# print(year_block)
# print(year_all[3:4])
# year_all2=year_all[11:]
# print(range(1,len(year_all2)))
# print(year_all2)

In [ ]:
# year_all2=year_all[3:4] # for 1965-1969 historical
# year_all2=year_all[0:1] # for 2006-2010 only
# year_all2=year_all[11:] # for 2046-2100
# year_all2=year_all[3:]
# for year_block in range(1,len(year_all2)): # for future (for now, will go back and get 2006-2010 and 1950-1954)

# This is the meat of the code: I kept year_all2 above because I had to reprocess some years because of
# previously corrupt data in the data bucket (I think because of the mini-pangeo crash from earlier)

for year_block in range(0,len(year_all)): # for future (for now, will go back and get 2006-2010 and 1950-1954)
    # year_block_files = grepfxn(year_all[year_block],historical_all_files+rcp_all_files);print(year_block_files)
    year_block_files = grepfxn(year_all[year_block],rcp_all_files)
    
    print(year_block_files)

    bounds=[southmost,northmost,westmost,eastmost]

    rcp_pr = lapply_brick(grepfxn("pr",year_block_files), 'precipitation', model_files,tiffolder,data_source,to_clip=to_clip,bounds=bounds,pad_factor=pad_factor)
    # downwelling shortwave radiation
    rcp_rsds = lapply_brick(grepfxn("rsds",year_block_files), 'surface_downwelling_shortwave_flux_in_air', model_files,tiffolder,data_source,to_clip=to_clip,bounds=bounds,pad_factor=pad_factor)
    # maximum air temperature
    rcp_tasmax = lapply_brick(grepfxn("tasmax",year_block_files), 'air_temperature', model_files,tiffolder,data_source,to_clip=to_clip,bounds=bounds,pad_factor=pad_factor)
    # minimum air temperature
    rcp_tasmin = lapply_brick(grepfxn("tasmin",year_block_files), 'air_temperature', model_files,tiffolder,data_source,to_clip=to_clip,bounds=bounds,pad_factor=pad_factor)
    # Now repeat above for the rcp 8.5 model outputs below

    if(data_source == 'METDATA'):

        rcp_uas = lapply_brick(grepfxn("uas",year_block_files), 'eastward_wind', model_files,tiffolder,data_source,to_clip=to_clip,bounds=bounds,pad_factor=pad_factor)

        rcp_vas = lapply_brick(grepfxn("vas",year_block_files), 'northward_wind', model_files,tiffolder,data_source,to_clip=to_clip,bounds=bounds,pad_factor=pad_factor)

        rcp_rhsmax = lapply_brick(grepfxn("rhsmax",year_block_files), 'relative_humidity', model_files,tiffolder,data_source,to_clip=to_clip,bounds=bounds,pad_factor=pad_factor)

        rcp_rhsmin = lapply_brick(grepfxn("rhsmin",year_block_files), 'relative_humidity', model_files,tiffolder,data_source,to_clip=to_clip,bounds=bounds,pad_factor=pad_factor)

        # The section below is meant to convert netCDF files into geoTIFFs
        
    src = rio.open(elevfile)
    # elevation_full_aggregate = aggregate_raster_inmem(src,scale=0.5)
    aggoutput_name='elevation_aggregated.tif'
    
    resample_raster_write(src, name= aggoutput_name,scale=0.5)
    
    dst_filename='elevation_aggregated_resampled.tif'
    match_filename=rcp_pr[0][0].name
    reproject_raster(aggoutput_name, match_filename,dst_filename)
    
    elevation_array=rio.open(dst_filename).read(1)

    # from datetime import datetime - will need to update to make start/end adaptive (7/28/2020)
#     start_year=year_all[year_block][0:4]
#     end_year=year_all[year_block][5:9]
    start_year=year_all[year_block][0:4]
    end_year=year_all[year_block][5:9]

    start=start_year+'-01-01'
    end=end_year+'-12-31'
    datetimes = pd.date_range(start=start,end=end)
    i=10
    
    for i in range(0,rcp_pr[0][0].count):

        doy_loop = pd.Period(datetimes[i],freq='D').dayofyear
        year_loop = pd.Period(datetimes[i],freq='D').year

        # step 1: extract ith band from the raster stack
        # step 2: stack those ith bands together
        # step 3: do raster mean math from step 2
        pr_stack=[]

        # Purpose: create stacks of variables individually - this is like brick in R
        pr_ensemble = []
        rsds_ensemble = []
        tasmax_ensemble = []
        tasmin_ensemble = []

        j = 0

        # should be 1 array for each variable (mean of x ensembles for a given doy)
        # rcp_pr[0][0].read(1, masked=False).shape
        rcp_pr_doy = rastermath(rcp_pr[0], i)
        rcp_rsds_doy = rastermath(rcp_rsds[0], i)
        rcp_tasmax_doy = rastermath(rcp_tasmax[0], i)
        rcp_tasmin_doy = rastermath(rcp_tasmin[0], i)

        dims = np.shape(rcp_pr_doy[0])
        rows = dims[0]
        cols = dims[1]
        constant_1_dat = np.full((rows,cols), 17.27)
        constant_2_dat = np.full((rows,cols), 0.6108)
        constant_3_dat = np.full((rows,cols), 273.15)
        constant_4_dat = np.full((rows,cols), 237.3)

        rcp_vs_tmax_array = constant_2_dat * np.exp(constant_1_dat * (rcp_tasmax_doy[0]-constant_3_dat) / ( (rcp_tasmax_doy[0]-constant_3_dat) + constant_4_dat)) # Equation S2.5
        rcp_vs_tmin_array = constant_2_dat * np.exp(constant_1_dat * (rcp_tasmin_doy[0]-constant_3_dat) / ( (rcp_tasmin_doy[0]-constant_3_dat) + constant_4_dat)) # Equation S2.5
        rcp_saturatedvapor_doy = (rcp_vs_tmax_array + rcp_vs_tmin_array)/2 

        if(data_source == 'METDATA'): # line 180 from R script

            # All of these are arrays by the way
            rcp_rhsmax_doy = rastermath(rcp_rhsmax[0], i)
            rcp_rhsmin_doy = rastermath(rcp_rhsmin[0], i)
            rcp_uas_doy = rastermath(rcp_uas[0], i)
            rcp_vas_doy = rastermath(rcp_vas[0], i)

            # was below are just arrays, not metadata profiles
            rcp_was_doy_10m = np.sqrt(rcp_uas_doy[0]**2 + rcp_vas_doy[0]**2 )

            rcp_actualvapor_doy = (rcp_vs_tmin_array * rcp_rhsmax_doy[0]/100 + rcp_vs_tmax_array * rcp_rhsmin_doy[0]/100)/2
            
#             rcp_actualvapor_doy = rcp_vs_tmin_array
            
#         elif (data_source == "Future_LIVNEH"):

#             pressure = atmospheric_pressure(elevation_array)

#             rcp_huss_doy = rastermath(rcp_huss[0], i)

#             rcp_was_doy_10m = rastermath(rcp_was[0], i)

#             rcp_tasmean_doy = (rcp_tasmax_doy[0] + rcp_tasmin_doy[0])/2

#             rcp_rhmean_doy = relative_fromspecific(pressure,rcp_tasmean_doy, rcp_huss_doy[0])

#             rcp_actualvapor_doy = rcp_rhmean_doy / 100 * rcp_saturatedvapor_doy

        da = xr.open_rasterio(rcp_pr[1])
        da_r = rio.open(rcp_pr[1])
        ny, nx = len(da['y']), len(da['x'])
        longitude_array, latitude_array = np.meshgrid(da['x'], da['y'])

        latitude_array_rad = latitude_array * (math.pi/180)

        # Wind speed at 2 meters
        z = np.full((rows,cols), 10)
        array_487 = np.full((rows,cols), 4.87)
        array_678 = np.full((rows,cols), 67.8)
        array_542 = np.full((rows,cols), 5.42)

        if (data_source == 'METDATA'):
            rcp_was_doy_2m = rcp_was_doy_10m * array_487 / np.log(array_678*z - array_542) # Equation S5.20 for PET formulations other than Penman
        else:
            rcp_was_doy_2m = rcp_was_doy_10m[0] * array_487 / np.log(array_678*z - array_542) # Equation S5.20 for PET formulations other than Penman

        doy_array = np.full((rows,cols), i+1)

        rcp_pr_doy[1]['count']=1
        rcp_tasmin_doy[1]['count']=1
        rcp_tasmax_doy[1]['count']=1

        # To-do: go ahead and developed ET0 directly as opposed to the R implementation(7/29)

        ET0_inputarrays_rcp = [rcp_pr_doy[0], rcp_rsds_doy[0], rcp_tasmin_doy[0],
                                     rcp_tasmax_doy[0],rcp_was_doy_2m,rcp_saturatedvapor_doy,
                                     rcp_actualvapor_doy,elevation_array,latitude_array_rad,doy_array]

        # NameError: name 'ET0_method' is not defined
        if ET0_method  == "yes":
          if ET0_crop != "short" and ET0_crop != "tall":
            stop("Please enter 'short' or 'tall' for the desired reference crop type")
          else:
            alpha = 0.23 # albedo for both short and tall crop
            if (ET0_crop == "short"):
              z0 = 0.02 # roughness height for short grass
            else:
              z0 = 0.1 # roughness height for tall grass
        else:
            z0 = 0.02 # roughness height for short grass
            alpha = 0.25 # semi-desert short grass - will not be used for calculation - just informative

        constants=[alpha, z0]

        ET0_rcp = ET0_PM(ET0_inputarrays_rcp,ET0_method,ET0_winddat,ET0_crop,constants)
        ET0_rcp.incoming_shortwave()
        ET0_rcp.outgoing_shortwave()
        ET0_rcp.outgoing_longwave()
        ET0_rcp.net_radiation()
        ET0_rcp_array_from_class = ET0_rcp.ET0_calcs()
        ET0_rcp_array_final = ET0_rcp_array_from_class.astype('float32')

        rcp_pr_doy[1]['count']=1

        os.chdir('/home/jupyter-rouze')
        gTIFF_filename = write_geotiff(data=ET0_rcp_array_final,meta=rcp_pr_doy[1],var_name='reference_evapotranspiration',
                                       doy=doy_loop,year=year_loop,folder=output_folder)

        #         local_file = root + '/' + gTIFF_filename 
        local_file = output_folder+'/' + 'reference_evapotranspiration' + '/' + gTIFF_filename 
        bucket = 'dev-et-data'
        #         bucket_filepath = 'in/DelawareRiverBasin/ETo/'+ gTIFF_filename 

#         if(os.path.isdir(str(year_loop))  == False):
#             root = output_folder+'/' + 'reference_evapotranspiration' + '/' + str(year_loop)
#             os.mkdir(root)
#         else:
#             root = output_folder+'/' + 'reference_evapotranspiration' + '/' + str(year_loop)

        bucket_filepath = 'in/DelawareRiverBasin/ETo/'+ str(year_loop)  + '/' + gTIFF_filename 
#         local_file = output_file+'/reference'
        
        os.chdir('/home/jupyter-rouze')
        s3_push_delete_local(local_file, bucket, bucket_filepath)


In [ ]:
# The code below is trash code that is either not used in the for loop above, or 
# is piecemeal like above. this stuff below should be ignored for inspection purposes (Gregory Rouze, 9/10/2020)

In [ ]:
print(year_all2)
print(year_all[year_block])
print(end_year)
print(datetimes)
print(year_loop)
print(gTIFF_filename)
print(bucket_filepath)

In [ ]:
        local_file = output_folder+'/' + 'reference_evapotranspiration' + '/' + gTIFF_filename 
        bucket = 'dev-et-data'
        #         bucket_filepath = 'in/DelawareRiverBasin/ETo/'+ gTIFF_filename 

#         if(os.path.isdir(str(year_loop))  == False):
#             root = output_folder+'/' + 'reference_evapotranspiration' + '/' + str(year_loop)
#             os.mkdir(root)
#         else:
#             root = output_folder+'/' + 'reference_evapotranspiration' + '/' + str(year_loop)

        bucket_filepath = 'in/DelawareRiverBasin/ETo/'+ str(year_loop)  + '/' + gTIFF_filename 
#         local_file = output_file+'/reference'

print(local_file)
print(bucket)
print(bucket_filepath)

In [ ]:
root = output_folder+'/' + 'reference_evapotranspiration' + '/' + str(year_loop)
print(root + '/' + gTIFF_filename)

In [ ]:
print(gTIFF_filename)
print('in/DelawareRiverBasin/ETo/Daily/'+ gTIFF_filename )

In [ ]:
# To-do: Push these from local to aws s3 data bucket

In [ ]:
print('hello world')

In [ ]:
# for year_block in range(0,len(year_all)):
# year_block_files = grepfxn(year_all[year_block],historical_all_files+rcp_all_files);print(year_block_files)
year_block_files = grepfxn(year_all[year_block],rcp_all_files);print(year_block_files)

bounds=[southmost,northmost,westmost,eastmost]

rcp_pr = lapply_brick(grepfxn("pr",year_block_files), 'precipitation', model_files,tiffolder,data_source,to_clip=to_clip,bounds=bounds,pad_factor=pad_factor)
# downwelling shortwave radiation
rcp_rsds = lapply_brick(grepfxn("rsds",year_block_files), 'surface_downwelling_shortwave_flux_in_air', model_files,tiffolder,data_source,to_clip=to_clip,bounds=bounds,pad_factor=pad_factor)
# maximum air temperature
rcp_tasmax = lapply_brick(grepfxn("tasmax",year_block_files), 'air_temperature', model_files,tiffolder,data_source,to_clip=to_clip,bounds=bounds,pad_factor=pad_factor)
# minimum air temperature
rcp_tasmin = lapply_brick(grepfxn("tasmin",year_block_files), 'air_temperature', model_files,tiffolder,data_source,to_clip=to_clip,bounds=bounds,pad_factor=pad_factor)
# Now repeat above for the rcp 8.5 model outputs below

if(data_source == 'METDATA'):

    rcp_uas = lapply_brick(grepfxn("uas",year_block_files), 'eastward_wind', model_files,tiffolder,data_source,to_clip=to_clip,bounds=bounds,pad_factor=pad_factor)

    rcp_vas = lapply_brick(grepfxn("vas",year_block_files), 'northward_wind', model_files,tiffolder,data_source,to_clip=to_clip,bounds=bounds,pad_factor=pad_factor)

    rcp_rhsmax = lapply_brick(grepfxn("rhsmax",year_block_files), 'relative_humidity', model_files,tiffolder,data_source,to_clip=to_clip,bounds=bounds,pad_factor=pad_factor)

    rcp_rhsmin = lapply_brick(grepfxn("rhsmin",year_block_files), 'relative_humidity', model_files,tiffolder,data_source,to_clip=to_clip,bounds=bounds,pad_factor=pad_factor)

    # The section below is meant to convert netCDF files into geoTIFFs


In [ ]:
print(rcp_uas)

In [ ]:
src = rio.open(elevfile)
# elevation_full_aggregate = aggregate_raster_inmem(src,scale=0.5)
aggoutput_name='elevation_aggregated.tif'
print(src)

In [ ]:
resample_raster_write(src, name= aggoutput_name,scale=0.5)

In [ ]:
dst_filename='elevation_aggregated_resampled.tif'
match_filename=rcp_pr[0][0].name
reproject_raster(aggoutput_name, match_filename,dst_filename)

In [ ]:
elevation_array=rio.open(dst_filename).read(1)

# from datetime import datetime - will need to update to make start/end adaptive (7/28/2020)
start_year=year_all[year_block][0:4]
end_year=year_all[year_block][5:9]

start=start_year+'-01-01'
end=end_year+'-12-31'
datetimes = pd.date_range(start=start,end=end)
i=10

In [ ]:
print(rcp_pr[0][0])
raster_array = rcp_pr[0][0].read(1, masked=False)

et = np.rot90(raster_array,2)
et = np.flip(et,1)

c1 = plt.imshow(et)
plt.colorbar(c1) 
plt.show()

In [ ]:
# for i in range(0,rcp_pr[0][0].count):

doy_loop = pd.Period(datetimes[i],freq='D').dayofyear
year_loop = pd.Period(datetimes[i],freq='D').year

# step 1: extract ith band from the raster stack
# step 2: stack those ith bands together
# step 3: do raster mean math from step 2
pr_stack=[]

# Purpose: create stacks of variables individually - this is like brick in R
pr_ensemble = []
rsds_ensemble = []
tasmax_ensemble = []
tasmin_ensemble = []

j = 0

# should be 1 array for each variable (mean of x ensembles for a given doy)
# rcp_pr[0][0].read(1, masked=False).shape
rcp_pr_doy = rastermath(rcp_pr[0], i)
rcp_rsds_doy = rastermath(rcp_rsds[0], i)
rcp_tasmax_doy = rastermath(rcp_tasmax[0], i)
rcp_tasmin_doy = rastermath(rcp_tasmin[0], i)

In [ ]:
print(rcp_tasmax_doy)
c1 = plt.imshow(rcp_tasmax_doy[0])
plt.colorbar(c1) 
plt.show()

In [ ]:
dims = np.shape(rcp_pr_doy[0])
rows = dims[0]
cols = dims[1]
constant_1_dat = np.full((rows,cols), 17.27)
constant_2_dat = np.full((rows,cols), 0.6108)
constant_3_dat = np.full((rows,cols), 273.15)
constant_4_dat = np.full((rows,cols), 237.3)

rcp_vs_tmax_array = constant_2_dat * np.exp(constant_1_dat * (rcp_tasmax_doy[0]-constant_3_dat) / ( (rcp_tasmax_doy[0]-constant_3_dat) + constant_4_dat)) # Equation S2.5
rcp_vs_tmin_array = constant_2_dat * np.exp(constant_1_dat * (rcp_tasmin_doy[0]-constant_3_dat) / ( (rcp_tasmin_doy[0]-constant_3_dat) + constant_4_dat)) # Equation S2.5
rcp_saturatedvapor_doy = (rcp_vs_tmax_array + rcp_vs_tmin_array)/2 

In [ ]:
if(data_source == 'METDATA'): # line 180 from R script

    # All of these are arrays by the way
    rcp_rhsmax_doy = rastermath(rcp_rhsmax[0], i)
    rcp_rhsmin_doy = rastermath(rcp_rhsmin[0], i)
    rcp_uas_doy = rastermath(rcp_uas[0], i)
    rcp_vas_doy = rastermath(rcp_vas[0], i)

    # was below are just arrays, not metadata profiles
    rcp_was_doy_10m = np.sqrt(rcp_uas_doy[0]**2 + rcp_vas_doy[0]**2 )

    rcp_actualvapor_doy = (rcp_vs_tmin_array * rcp_rhsmax_doy[0]/100 + rcp_vs_tmax_array * rcp_rhsmin_doy[0]/100)/2

elif (data_source == "Future_LIVNEH"):

    pressure = atmospheric_pressure(elevation_array)

    rcp_huss_doy = rastermath(rcp_huss[0], i)

    rcp_was_doy_10m = rastermath(rcp_was[0], i)

    rcp_tasmean_doy = (rcp_tasmax_doy[0] + rcp_tasmin_doy[0])/2

    rcp_rhmean_doy = relative_fromspecific(pressure,rcp_tasmean_doy, rcp_huss_doy[0])

    rcp_actualvapor_doy = rcp_rhmean_doy / 100 * rcp_saturatedvapor_doy

In [ ]:
rcp_was_doy_10m

In [ ]:
print(rcp_pr)
xr.open_rasterio(rcp_pr[1])

In [ ]:
# Compute the lon/lat coordinates with rasterio.warp.transform
# fs = fsspec.filesystem('s3')
# fobj = fs.open(original_list[j])
# xds = xr.open_dataset(fobj)

da = xr.open_rasterio(rcp_pr[1])
da_r = rio.open(rcp_pr[1])
ny, nx = len(da['y']), len(da['x'])
longitude_array, latitude_array = np.meshgrid(da['x'], da['y'])

latitude_array_rad = latitude_array * (math.pi/180)

# Wind speed at 2 meters
z = np.full((rows,cols), 10)
array_487 = np.full((rows,cols), 4.87)
array_678 = np.full((rows,cols), 67.8)
array_542 = np.full((rows,cols), 5.42)

In [ ]:
if (data_source == 'METDATA'):
    rcp_was_doy_2m = rcp_was_doy_10m * array_487 / np.log(array_678*z - array_542) # Equation S5.20 for PET formulations other than Penman
else:
    rcp_was_doy_2m = rcp_was_doy_10m[0] * array_487 / np.log(array_678*z - array_542) # Equation S5.20 for PET formulations other than Penman

doy_array = np.full((rows,cols), i+1)

rcp_pr_doy[1]['count']=1
rcp_tasmin_doy[1]['count']=1
rcp_tasmax_doy[1]['count']=1

In [ ]:
# technically speaking, writing these arrays as geotiffs is optional (except precip), as technically only precip and ET0 are needed - however, they provide for good sanity checks.
# write_geotiff(data=rcp_pr_doy[0],meta=rcp_pr_doy[1],var_name='precipitation',doy=doy_loop,year=year_loop,folder=output_folder)
# write_geotiff(data=rcp_tasmin_doy[0],meta=rcp_tasmin_doy[1],var_name='airtemperature_min',doy=doy_loop,year=year_loop,folder=output_folder)
# write_geotiff(data=rcp_tasmax_doy[0],meta=rcp_tasmax_doy[1],var_name='airtemperature_max',doy=doy_loop,year=year_loop,folder=output_folder)


In [ ]:

# To-do: go ahead and developed ET0 directly as opposed to the R implementation(7/29)

ET0_inputarrays_rcp = [rcp_pr_doy[0], rcp_rsds_doy[0], rcp_tasmin_doy[0],
                             rcp_tasmax_doy[0],rcp_was_doy_2m,rcp_saturatedvapor_doy,
                             rcp_actualvapor_doy,elevation_array,latitude_array_rad,doy_array]

# NameError: name 'ET0_method' is not defined
if ET0_method  == "yes":
  if ET0_crop != "short" and ET0_crop != "tall":
    stop("Please enter 'short' or 'tall' for the desired reference crop type")
  else:
    alpha = 0.23 # albedo for both short and tall crop
    if (ET0_crop == "short"):
      z0 = 0.02 # roughness height for short grass
    else:
      z0 = 0.1 # roughness height for tall grass
else:
    z0 = 0.02 # roughness height for short grass
    alpha = 0.25 # semi-desert short grass - will not be used for calculation - just informative


In [ ]:
import matplotlib.pyplot as plt
# print(ET0_inputarrays_rcp[1])
test=ET0_inputarrays_rcp[1]
print(test)
c1 = plt.imshow(test)
plt.colorbar(c1) 
plt.show()

In [ ]:
print(ET0_inputarrays_rcp)

In [ ]:
constants=[alpha, z0]

ET0_rcp = ET0_PM(ET0_inputarrays_rcp,ET0_method,ET0_winddat,ET0_crop,constants)
ET0_rcp.incoming_shortwave()
ET0_rcp.outgoing_shortwave()
ET0_rcp.outgoing_longwave()
ET0_rcp.net_radiation()
ET0_rcp_array_from_class = ET0_rcp.ET0_calcs()
ET0_rcp_array_final = ET0_rcp_array_from_class.astype('float32')

rcp_pr_doy[1]['count']=1


In [ ]:
type(ET0_rcp_array_final)
import matplotlib.pyplot as plt
c = plt.imshow(ET0_rcp_array_final,origin='bottom')
plt.colorbar(c) 
plt.show()

In [ ]:
print(os.getcwd())
os.chdir('in')

In [ ]:
os.chdir('/home/jupyter-rouze')
write_geotiff(data=ET0_rcp_array_final,meta=rcp_pr_doy[1],var_name='reference_evapotranspiration',
              doy=doy_loop,year=year_loop,folder=output_folder)


In [ ]:
os.chdir('/home/jupyter-rouze/in')
read = rio.open('reference_evapotranspiration/reference_evapotranspiration_2011011.tif')
array = read.read(1)
c = plt.imshow(array, origin='lower')
plt.colorbar(c) 
plt.show()
# from rasterio.transform import Affine

# res = (array[-1] - array[0]) / 240.0
# transform = Affine.translation(array[0] - res / 2, array[0] - res / 2) * Affine.scale(res, res)

# from rasterio.plot import show
# show(array, transform=read.transform)
# import pyplot
# pyplot.imshow(array, origin='lower')
from rasterio import plot
plot.show(read, origin='lower', transform=read.transform)

In [ ]:
rio.plot.plotting_extent(read)

In [ ]:
original_list=grepfxn("uas",year_block_files)
var_name='eastward_wind'
model_files
tiffolder
data_source
to_clip=to_clip
bounds=bounds
pad_factor=pad_factor

print(to_clip)
fs = fsspec.filesystem('s3')
fobj = fs.open(original_list[0])
print(fobj)
xds = xr.open_dataset(fobj)

In [ ]:

final_list=[]
j=0

if((data_source == 'METDATA') or (data_source == 'gridMET') or (data_source == 'Historical1915-LIVNEH')):

    fs = fsspec.filesystem('s3')
    fobj = fs.open(original_list[j])
    xds = xr.open_dataset(fobj)

if((data_source == 'METDATA') or (data_source == 'gridMET')):
    orig_crs = xds.coordinate_system
else:
    print('hello world')

In [ ]:
netcdf_var=xds[var_name]
print(netcdf_var)
if(data_source == 'LIVNEHhistorical1915'):
    netcdf_var.rio.set_crs('EPSG:4326', inplace=True)

final = netcdf_var.assign_coords(lon=(((netcdf_var.lon + 180) % 360) - 180))

final.rio.set_spatial_dims(x_dim="lon", y_dim="lat")

# netcdf_var.assign_coords(lon=(((netcdf_var.lon + 180) % 360) - 180))

# netcdf_var.rio.set_spatial_dims(x_dim="lon", y_dim="lat")
# final.data=np.flip(final.data,1) # turned off on 8/17/2020 for the gridMET data

newfilename = original_list[j][:-3]+'.tif' # append .tif instead of .nc
os.chdir('/home/jupyter-rouze')
#             os.chdir('in/temp') # for local directory in mini-pangeo
# os.chdir('in/temp')
# os.getcwd()
# bounds=gridMET_pr[0][0].bounds

In [ ]:


if (to_clip == 'True'):

    lats = final.coords['lat'][:] 
    lons = final.coords['lon'][:]
    # lat_bnds, lon_bnds = [38.6, 42.54], [-76.3, -74.23] # originall hard-coded
    lat_bnds, lon_bnds = [bounds[0]-pad_factor, bounds[1]+pad_factor], [bounds[2]-pad_factor, bounds[3]+pad_factor]

    lat_inds = np.where((lats > lat_bnds[0]) & (lats < lat_bnds[1]))[0]
    lon_inds = np.where((lons > lon_bnds[0]) & (lons < lon_bnds[1]))[0]

    subset_netcdf=final[:,lat_inds,lon_inds]
    subset_netcdf.rio.set_spatial_dims(x_dim="lon", y_dim="lat")

    local_file = 'in/temp/'+ original_list[0][-63:][:-3]+'.tif'

    bucket = original_list[0][0:11]

    bucket_filepath = original_list[0][12:]

#                 os.getcwd()

    subset_netcdf.rio.to_raster(local_file)
    final_list.append(rio.open(local_file))


else:

    final.rio.to_raster(newfilename, driver='GTiff')

print(final_list)
    
#         fs.close(fobj)

#     s3_push_delete_local(local_file, bucket, bucket_filepath)

In [ ]:
# Loop by each time period block
# First, create a data frame with start/end years

# break down the name of each file into each metadata component
# The section below is meant to convert netCDF files into geoTIFFs and combine them in a brick like equivalent object similar to R

bounds=[southmost,northmost,westmost,eastmost]

# local_file = 'in/temp/'+ grepfxn("pr",year_block_files)[0][-61:][:-3]+'.tif'

# bucket = grepfxn("pr",year_block_files)[0][0:11]

# bucket_filepath = grepfxn("pr",year_block_files)[0][12:]

# final_list.append(rio.open(fs.open(newfilename)))

# fs.open(newfilename)

# print(local_file)
# print(bucket)
# print(bucket_filepath)
# print(grepfxn("pr",year_block_files))

# fobj.close()
fs = fsspec.filesystem('s3')
# grepfxn("rsds",year_block_files)[0]

# grepfxn("rsds",year_block_files)[0]
# fobj = fs.open(grepfxn("tasmax",year_block_files)[0])
# fobj2 = fs.open(grepfxn("tasmin",year_block_files)[0])
# fobj3 = fs.open(grepfxn("pr",year_block_files)[0])

# # print(type(fobj))
# # print(type(fobj2))

# print(fobj)
# print(fobj2)
# print(fobj3)

# # print(fobj)
# xds = xr.open_dataset(fobj)
# xds2 = xr.open_dataset(fobj2)
# xds3 = xr.open_dataset(fobj3)
# print(fobj)

# xds2 = xr.open_dataset(fobj2)

# print(xds2)

# type(fobj)
# fobj.close()
# fobj
# type(fobj)

# os.getcwd()

# # fs = fsspec.filesystem('s3')
# # fobj = fs.open(grepfxn("tasmin",year_block_files)[0])
# # print(fobj)
# # xds = xr.open_dataset(fobj)

# fs = fsspec.filesystem('s3')
# fobj2 = fs.open(grepfxn("uas",year_block_files)[0])
# print(fobj2)
# xds = xr.open_dataset(fobj2)

# local_file = 'in/temp/'+ grepfxn("tasmin",year_block_files)[0][-65:][:-3]+'.tif'
# print(local_file)
rcp_pr = lapply_brick(grepfxn("pr",year_block_files), 'precipitation', model_files,tiffolder,data_source,to_clip=to_clip,bounds=bounds,pad_factor=pad_factor)
# downwelling shortwave radiation
# rcp_rsds = lapply_brick(grepfxn("rsds",year_block_files), 'surface_downwelling_shortwave_flux_in_air', model_files,tiffolder,data_source,to_clip=to_clip,bounds=bounds,pad_factor=pad_factor)
# maximum air temperature
# rcp_tasmax = lapply_brick(grepfxn("tasmax",year_block_files), 'air_temperature', model_files,tiffolder,data_source,to_clip=to_clip,bounds=bounds,pad_factor=pad_factor)
# minimum air temperature
# rcp_tasmin = lapply_brick(grepfxn("tasmin",year_block_files), 'air_temperature', model_files,tiffolder,data_source,to_clip=to_clip,bounds=bounds,pad_factor=pad_factor)
# Now repeat above for the rcp 8.5 model outputs below

In [ ]:

local_file = 'in/temp/'+ grepfxn("uas",year_block_files)[-65:][0][:-3]+'.tif'
print(local_file)
# bucket = original_list[0][0:11]

# bucket_filepath = original_list[0][12:]

# #                 os.getcwd()

# subset_netcdf.rio.to_raster(local_file)

# s3_push_delete_local(local_file, bucket, bucket_filepath)

In [ ]:
if (data_source == "METDATA"):

    # Whittle down the number of files if the folder contains both rcp 4.5 and rcp 8.5 files
    # Right now, the code can only handle one model of METDATA output (8/21/2020)
    rcp_all_files = [grepfxn(rcp_source,all_files)][0]

    # Split models apart that are to be used for ensemble averaging
    models_parsed = [x.strip() for x in model.split(',')]

    # Iterate the files by each each specified model
    models_list=[]
    for i in range(len(models_parsed)):
        model_files_loop = [grepfxn(models_parsed[i],rcp_all_files)][0]
        models_list.append(model_files_loop)

    # Flatten series of lists into one list
    rcp_all_files = list(chain(*models_list))

    # Find and compile the year blocks into a list
    dfis=[]
    for out in rcp_all_files:
        a=out.split('_')
        dfi = a[5]+'_'+a[6]
        dfis.append(dfi)

    # Distill the above list into unique year blocks, as there will be duplicates from 
    # multiple climate inputs
    year_all=unique(dfis)

    # For prototyping only
    year_block=0
    for year_block in range(0,len(year_all)):

        year_block_files = grepfxn(year_all[year_block],rcp_all_files)
        # Loop by each time period block
        # First, create a data frame with start/end years

        # break down the name of each file into each metadata component
        # The section below is meant to convert netCDF files into geoTIFFs and combine them in a brick like equivalent object similar to R

        bounds=[southmost,northmost,westmost,eastmost]

        rcp_pr = lapply_brick(grepfxn("pr",year_block_files), 'precipitation', model_files,tiffolder,data_source,to_clip=to_clip,bounds=bounds,pad_factor=pad_factor)
        # downwelling shortwave radiation
        rcp_rsds = lapply_brick(grepfxn("rsds",year_block_files), 'surface_downwelling_shortwave_flux_in_air', model_files,tiffolder,data_source,to_clip=to_clip,bounds=bounds,pad_factor=pad_factor)
        # maximum air temperature
        rcp_tasmax = lapply_brick(grepfxn("tasmax",year_block_files), 'air_temperature', model_files,tiffolder,data_source,to_clip=to_clip,bounds=bounds,pad_factor=pad_factor)
        # minimum air temperature
        rcp_tasmin = lapply_brick(grepfxn("tasmin",year_block_files), 'air_temperature', model_files,tiffolder,data_source,to_clip=to_clip,bounds=bounds,pad_factor=pad_factor)
        # Now repeat above for the rcp 8.5 model outputs below

        if(data_source == 'METDATA'):

            rcp_uas = lapply_brick(grepfxn("uas",year_block_files), 'eastward_wind', model_files,tiffolder,data_source,to_clip=to_clip,bounds=bounds,pad_factor=pad_factor)

            rcp_vas = lapply_brick(grepfxn("vas",year_block_files), 'northward_wind', model_files,tiffolder,data_source,to_clip=to_clip,bounds=bounds,pad_factor=pad_factor)

            rcp_rhsmax = lapply_brick(grepfxn("rhsmax",year_block_files), 'relative_humidity', model_files,tiffolder,data_source,to_clip=to_clip,bounds=bounds,pad_factor=pad_factor)

            rcp_rhsmin = lapply_brick(grepfxn("rhsmin",year_block_files), 'relative_humidity', model_files,tiffolder,data_source,to_clip=to_clip,bounds=bounds,pad_factor=pad_factor)

        # The section below is meant to convert netCDF files into geoTIFFs
        # and combine them in a brick like equivalent object similar to R

    ############################################# This ends the read nc file and stack all temporal bands

        src = rio.open(elevfile)
        # elevation_full_aggregate = aggregate_raster_inmem(src,scale=0.5)
        aggoutput_name='elevation_aggregated.tif'
        resample_raster_write(src, name= aggoutput_name,scale=0.5)
        dst_filename='elevation_aggregated_resampled.tif'
        match_filename=rcp_pr[0][0].name
        reproject_raster(aggoutput_name, match_filename,dst_filename)

        elevation_array=rio.open(dst_filename).read(1)

        # from datetime import datetime - will need to update to make start/end adaptive (7/28/2020)
        start_year=year_all[year_block][0:4]
        end_year=year_all[year_block][5:9]

        start=start_year+'-01-01'
        end=end_year+'-12-31'
        datetimes = pd.date_range(start=start,end=end)
        i=10
        for i in range(0,rcp_pr[0][0].count):

            doy_loop = pd.Period(datetimes[i],freq='D').dayofyear
            year_loop = pd.Period(datetimes[i],freq='D').year

            # step 1: extract ith band from the raster stack
            # step 2: stack those ith bands together
            # step 3: do raster mean math from step 2
            pr_stack=[]

            # Purpose: create stacks of variables individually - this is like brick in R
            pr_ensemble = []
            rsds_ensemble = []
            tasmax_ensemble = []
            tasmin_ensemble = []

            j = 0

            # should be 1 array for each variable (mean of x ensembles for a given doy)
            # rcp_pr[0][0].read(1, masked=False).shape
            rcp_pr_doy = rastermath(rcp_pr[0], i)
            rcp_rsds_doy = rastermath(rcp_rsds[0], i)
            rcp_tasmax_doy = rastermath(rcp_tasmax[0], i)
            rcp_tasmin_doy = rastermath(rcp_tasmin[0], i)

            dims = np.shape(rcp_pr_doy[0])
            rows = dims[0]
            cols = dims[1]
            constant_1_dat = np.full((rows,cols), 17.27)
            constant_2_dat = np.full((rows,cols), 0.6108)
            constant_3_dat = np.full((rows,cols), 273.15)
            constant_4_dat = np.full((rows,cols), 237.3)

            rcp_vs_tmax_array = constant_2_dat * np.exp(constant_1_dat * (rcp_tasmax_doy[0]-constant_3_dat) / ( (rcp_tasmax_doy[0]-constant_3_dat) + constant_4_dat)) # Equation S2.5
            rcp_vs_tmin_array = constant_2_dat * np.exp(constant_1_dat * (rcp_tasmin_doy[0]-constant_3_dat) / ( (rcp_tasmin_doy[0]-constant_3_dat) + constant_4_dat)) # Equation S2.5
            rcp_saturatedvapor_doy = (rcp_vs_tmax_array + rcp_vs_tmin_array)/2 

            if(data_source == 'METDATA'): # line 180 from R script

                # All of these are arrays by the way
                rcp_rhsmax_doy = rastermath(rcp_rhsmax[0], i)
                rcp_rhsmin_doy = rastermath(rcp_rhsmin[0], i)
                rcp_uas_doy = rastermath(rcp_uas[0], i)
                rcp_vas_doy = rastermath(rcp_vas[0], i)

                # was below are just arrays, not metadata profiles
                rcp_was_doy_10m = np.sqrt(rcp_uas_doy[0]**2 + rcp_vas_doy[0]**2 )

                rcp_actualvapor_doy = (rcp_vs_tmin_array * rcp_rhsmax_doy[0]/100 + rcp_vs_tmax_array * rcp_rhsmin_doy[0]/100)/2

            elif (data_source == "Future_LIVNEH"):

                pressure = atmospheric_pressure(elevation_array)

                rcp_huss_doy = rastermath(rcp_huss[0], i)

                rcp_was_doy_10m = rastermath(rcp_was[0], i)

                rcp_tasmean_doy = (rcp_tasmax_doy[0] + rcp_tasmin_doy[0])/2

                rcp_rhmean_doy = relative_fromspecific(pressure,rcp_tasmean_doy, rcp_huss_doy[0])

                rcp_actualvapor_doy = rcp_rhmean_doy / 100 * rcp_saturatedvapor_doy

            # Compute the lon/lat coordinates with rasterio.warp.transform
            da = xr.open_rasterio(rcp_pr[1])
            da_r = rio.open(rcp_pr[1])
            ny, nx = len(da['y']), len(da['x'])
            longitude_array, latitude_array = np.meshgrid(da['x'], da['y'])

            latitude_array_rad = latitude_array * (math.pi/180)

            # Wind speed at 2 meters
            z = np.full((rows,cols), 10)
            array_487 = np.full((rows,cols), 4.87)
            array_678 = np.full((rows,cols), 67.8)
            array_542 = np.full((rows,cols), 5.42)

            if (data_source == 'METDATA'):
                rcp_was_doy_2m = rcp_was_doy_10m * array_487 / np.log(array_678*z - array_542) # Equation S5.20 for PET formulations other than Penman
            else:
                rcp_was_doy_2m = rcp_was_doy_10m[0] * array_487 / np.log(array_678*z - array_542) # Equation S5.20 for PET formulations other than Penman

            doy_array = np.full((rows,cols), i+1)

            rcp_pr_doy[1]['count']=1
            rcp_tasmin_doy[1]['count']=1
            rcp_tasmax_doy[1]['count']=1

            # technically speaking, writing these arrays as geotiffs is optional (except precip), as technically only precip and ET0 are needed - however, they provide for good sanity checks.
            write_geotiff(data=rcp_pr_doy[0],meta=rcp_pr_doy[1],var_name='precipitation',doy=doy_loop,year=year_loop,folder=output_folder)
            write_geotiff(data=rcp_tasmin_doy[0],meta=rcp_tasmin_doy[1],var_name='airtemperature_min',doy=doy_loop,year=year_loop,folder=output_folder)
            write_geotiff(data=rcp_tasmax_doy[0],meta=rcp_tasmax_doy[1],var_name='airtemperature_max',doy=doy_loop,year=year_loop,folder=output_folder)

            # To-do: go ahead and developed ET0 directly as opposed to the R implementation(7/29)

            ET0_inputarrays_rcp = [rcp_pr_doy[0], rcp_rsds_doy[0], rcp_tasmin_doy[0],
                                         rcp_tasmax_doy[0],rcp_was_doy_2m,rcp_saturatedvapor_doy,
                                         rcp_actualvapor_doy,elevation_array,latitude_array_rad,doy_array]

            # NameError: name 'ET0_method' is not defined
            if ET0_method  == "yes":
              if ET0_crop != "short" and ET0_crop != "tall":
                stop("Please enter 'short' or 'tall' for the desired reference crop type")
              else:
                alpha = 0.23 # albedo for both short and tall crop
                if (ET0_crop == "short"):
                  z0 = 0.02 # roughness height for short grass
                else:
                  z0 = 0.1 # roughness height for tall grass
            else:
                z0 = 0.02 # roughness height for short grass
                alpha = 0.25 # semi-desert short grass - will not be used for calculation - just informative

            constants=[alpha, z0]

            ET0_rcp = ET0_PM(ET0_inputarrays_rcp,ET0_method,ET0_winddat,ET0_crop,constants)
            ET0_rcp.incoming_shortwave()
            ET0_rcp.outgoing_shortwave()
            ET0_rcp.outgoing_longwave()
            ET0_rcp.net_radiation()
            ET0_rcp_array_from_class = ET0_rcp.ET0_calcs()
            ET0_rcp_array_final = ET0_rcp_array_from_class.astype('float32')

            rcp_pr_doy[1]['count']=1
            os.chdir(output_folder)
            write_geotiff(data=ET0_rcp_array_final,meta=rcp_pr_doy[1],var_name='reference_evapotranspiration',
                          doy=doy_loop,year=year_loop,folder=output_folder)
